In [17]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName('load').enableHiveSupport().getOrCreate()

In [28]:
def autenticar():
    # endpoint da API para autenticar
    api_url = "https://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=fa1ae741481d20625673b2020fdd07bcfdcf5d60f27d226a069812a94de3edd0"

    # Inicialize uma sessão do requests
    session = requests.Session()

    try:
        # Faz a requisição post para a API usando a sessão
        response = session.post(api_url)

        # Verifique se a requisição foi bem-sucedida (código de status 200)
        if response.status_code == 200:
            # Extraia o conteúdo JSON (ou texto) da resposta
            data = response.json()  # ou response.text se for texto
          
            return session                
        else:
            print(f"Falha ao acessar a API. Status Code: {session.post(api_url).status_code}")      
            
    finally:
        # Feche a sessão após o uso
        session.close()

def callAPIGet(api_url, session):
    try:    
        # Verifique se a requisição foi bem-sucedida (código de status 200)
        if session.get(api_url).status_code == 200:
            # Extraia o conteúdo JSON (ou texto) da resposta
            data = session.get(api_url).json()  # ou response.text se for texto
            
            if isinstance(data, list):
                return data
            elif isinstance(data, dict):
                return [data]
            
            return None                
        else:
            print(f"Falha ao acessar a API. Status Code: {session.post(api_url).status_code}")      
            
    finally:
        # Feche a sessão após o uso
        session.close()

In [29]:
corredor_schema = StructType([
    StructField("cc", IntegerType(), True),
    StructField("nc", StringType(), True)
])

empresa_schema = StructType([
    StructField("hr", StringType(), True),
    StructField("e", ArrayType(StructType([
        StructField("a", IntegerType(), True),
        StructField("e", ArrayType(StructType([
            StructField("a", IntegerType(), True),
            StructField("c", IntegerType(), True),
            StructField("n", StringType(), True)
        ])), True),
    ])), True)
])

posicao_schema = StructType([
    StructField("hr", StringType(), True),
    StructField("l", ArrayType(StructType([
        StructField("c", StringType(), True),
        StructField("cl", IntegerType(), True),
        StructField("sl", IntegerType(), True),
        StructField("lt0", StringType(), True),
        StructField("lt1", StringType(), True),
        StructField("qv", IntegerType(), True),
        StructField("vs", ArrayType(StructType([
            StructField("p", IntegerType(), True),
            StructField("a", BooleanType(), True),
            StructField("ta", StringType(), True),
            StructField("py", FloatType(), True),
            StructField("px", FloatType(), True)
        ])), True),
    ])), True),
])

In [33]:
# linhas = spark.read.csv("s3a://raw/GTFS/linhas/*.txt", header=True)

session = autenticar()

corredor = callAPIGet("https://api.olhovivo.sptrans.com.br/v2.1/Corredor", session)
# print(corredor)
corredor_df = spark.createDataFrame(corredor, corredor_schema)
# corredor_df.write.mode("append").format("json").save("s3a://raw/olhovivo/corredor/")

empresa = callAPIGet("https://api.olhovivo.sptrans.com.br/v2.1/Empresa", session)
# print(empresa)
empresa_df = spark.createDataFrame(empresa, empresa_schema)
# empresa_df.write.mode("append").format("json").save("s3a://raw/olhovivo/empresa/")

posicao = callAPIGet("https://api.olhovivo.sptrans.com.br/v2.1/Posicao", session)
print(posicao)
posicao_df = spark.createDataFrame(posicao, posicao_schema)
# posicao_df.write.mode("append").format("json").save("s3a://raw/olhovivo/Posicao/")


# Previsão de chegada

[{'hr': '18:13', 'l': [{'c': '7016-10', 'cl': 34886, 'sl': 2, 'lt0': 'TERM. STO. AMARO', 'lt1': 'JD. ÂNGELA', 'qv': 5, 'vs': [{'p': 73471, 'a': True, 'ta': '2024-09-15T21:13:10Z', 'py': -23.6613095, 'px': -46.707041, 'sv': None, 'is': None}, {'p': 73431, 'a': True, 'ta': '2024-09-15T21:13:16Z', 'py': -23.6882555, 'px': -46.7733525, 'sv': None, 'is': None}, {'p': 73482, 'a': True, 'ta': '2024-09-15T21:13:07Z', 'py': -23.6886205, 'px': -46.77281625, 'sv': None, 'is': None}, {'p': 73448, 'a': True, 'ta': '2024-09-15T21:13:25Z', 'py': -23.665758, 'px': -46.7126025, 'sv': None, 'is': None}, {'p': 73447, 'a': True, 'ta': '2024-09-15T21:12:57Z', 'py': -23.6910715, 'px': -46.7607575, 'sv': None, 'is': None}]}, {'c': '2731-10', 'cl': 946, 'sl': 1, 'lt0': 'METRÔ ITAQUERA', 'lt1': 'VL. PROGRESSO', 'qv': 5, 'vs': [{'p': 35779, 'a': True, 'ta': '2024-09-15T21:13:14Z', 'py': -23.518249, 'px': -46.439041, 'sv': None, 'is': None}, {'p': 35744, 'a': True, 'ta': '2024-09-15T21:11:21Z', 'py': -23.5419734